## Part4 Visualization

### ライブラリのインポート

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pyl
from matplotlib.colors import LinearSegmentedColormap
import plotly.express as px
import json

### 前のパートで作成した、DataFrameをロード

In [2]:
df = pd.read_csv('./df_analysis.csv')
df.loc[:,'JCODE'] = df.loc[:,'JCODE'].apply(str).str.rjust(5,'0')
df.set_index('JCODE',inplace=True)
df.loc[:,'Class'] = df.loc[:,'Class'].apply(str)

### シェープファイルから市区町村のジオメトリーポリゴンデータをロード

In [3]:
path_shapefile = "./japan_ver81/japan_ver81.shp"
df_gis = gpd.read_file(path_shapefile, encoding="SHIFT-JIS")
list_field = ['geometry','KEN']
df_gis.set_index('JCODE',inplace=True)
df_gis = df_gis.loc[pd.isnull(df_gis.index) == False,list_field]
df_gis = df_gis.loc[df_gis.index.drop_duplicates()]

### クラスター分類のクラス名を凡例用に変更

In [4]:
def rename_class(df_concat):
    df_concat.sort_values(by=['Class'],inplace=True)
    num_class = len(df_concat.loc[:,'Class'].unique())
    for i in range(num_class):
        idx_rename = df_concat.loc[:,'Class'] == str(i)        
        if i == 0:
            df_concat.loc[idx_rename,'Class'] = 'N/A'
        else:
            df_concat.loc[idx_rename,'Class'] = 'Class '+ str(i)
    return df_concat

In [5]:
df_concat = pd.concat([df_gis,df], axis=1, sort=False)
df_concat.loc[pd.isnull(df_concat['Class']) == True,'Class'] = str(0)
df_concat.reset_index(inplace=True)
df_concat.rename(columns={'index':'JCODE'},inplace=True)
df_concat = rename_class(df_concat)

### 地域の定義

In [6]:
list_ken = list(df_gis.KEN.unique())
list_reg = ['北海道','東北','関東','中部','近畿','中国','四国','九州','沖縄']
dict_reg = {list_reg[0]:[list_ken[0]],list_reg[1]:list_ken[1:7],list_reg[2]:list_ken[7:14],
            list_reg[3]:list_ken[14:23],list_reg[4]:list_ken[23:30],list_reg[5]:list_ken[30:35],
            list_reg[6]:list_ken[35:39],list_reg[7]:list_ken[39:46],list_reg[8]:[list_ken[46]]}

# 例えば近畿地方の市区町村のみを表示させる場合、以下の通り。以下の2行をコメントアウトすれば、全国の地図が表示される。
name_region = list_reg[8]
df_show = df_concat.loc[df_concat.loc[:,'KEN'].isin(dict_reg[name_region]),:]

### Plotly Expressにより、インタラクティブな表示を行う。

In [7]:
def plot_choropleth(df,geojson):
    # Plotly Expressで表示
    fig = px.choropleth(df,geojson=geojson,color="Class",locations="JCODE",
                        featureidkey="properties.JCODE", projection="mercator",
                        hover_data=["KEN","CITY_JP","Class"])
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [8]:
# ジオメトリーデータをgeojsonファイルとして保存し、読み込み。
path_geojson = "./japan_ver81/japan.geojson"
list_field = ['JCODE','CITY_ENG','Class']
df_show.loc[:,['JCODE','geometry']].to_file(path_geojson, driver='GeoJSON')
with open(path_geojson) as json_file:
    geojson = json.load(json_file)

In [ ]:
# ファイルサイズが大きいため、非表示。実際に作成された図はfigureフォルダに保存。
plot_choropleth(df_show,geojson)

Class1: 持ち家ありの家族世帯、治安良し、車社会、人口減少 <br>
Class2: 若い夫婦が多い郊外 <br>
Class3: あまり特徴なし、中規模の地方都市 <br>
Class4: 混沌とした大都会 <br>
Class5: ベッドタウン <br>
Class6: 過疎化が進んでいる中規模の地方都市 <br>
N/A:    データ不足のため、分析対象外、田舎